In [1]:
!pip install soundfile
!pip install librosa
!pip install audiomentations
!pip install tensorflow==2.3.0 --user

In [2]:
import os, random
import csv
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd
import librosa
import librosa.display
import numpy as np
import tensorflow as tf
import math
import shutil
import tensorflow.keras.backend as K

from audiomentations import Compose, AddGaussianNoise, AddGaussianSNR, FrequencyMask
from datetime import datetime
from matplotlib import image
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, Sequential
from tensorflow.python.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import losses
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

# Parameters

In [37]:
PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
ORIGINAL_DATASET_DIRECTORY = os.path.join(PROJECT_PATH, 'dataset', 'rfcx-species-audio-detection')
DATASET_DIRECTORY = os.path.join(PROJECT_PATH, 'dataset', 'spectrogram-species-audio-detection')
DATASET_TRAIN_DIRECTORY = os.path.join(DATASET_DIRECTORY, 'train')
DATASET_VAL_DIRECTORY = os.path.join(DATASET_DIRECTORY, 'val')
DATASET_TEST_DIRECTORY = os.path.join(DATASET_DIRECTORY, 'test')
WEIGHT_FILE_NAME = "EfficientNet_Weights/EfficientNetBN_tl_best_weights.h5"
IMAGE_HEIGHT = 500
IMAGE_WIDTH = 500

# Créer une 25eme classe qui ne correspond à aucun oiseau
USE_EMPTY_CLASS = True
len_classes = 25 if USE_EMPTY_CLASS else 24
epch = 100
KERNEL_REGULARIZERS = 0.0005
ref_lr = 0.03
ref_batch_size = 16
dropout = 0.2
batch_size = 2
momentumTest = 0.95
destination_classes = [str(i) for i in range(len_classes)]

### PARAMS spectrogramm_conversion ###
# Lié à IMAGE_WIDTH et IMAGE_HEIGHT
PERCENT_PRINT = 10
# duration_cut -> Découpage des extraits en morceaux de x secondes / 0 = pas de découpage
DURATION_CUT = 2
RANDOM_CUT = True
# Un ratio de 5 permet de sauvegarder 1 enregistrement de la 25eme classe sur 5
# Evite d'avoir une 25eme classe trop chargée en données (sachant que 1 enregistrement contient au minimum 2 extraits)
RATIO_EMPTY_CLASS = 40
PRED_EMPTY_IGNORE_EXTRACT = 0.6
# minimum duration of record
MINIMAL_DURATION = 0.25
MINIMAL_ANIMAL_PRESENCE = 0.25
FREQ_MODIFIER = 0
validation_split = 0.3
USE_DATA_AUGMENTATION = False
RATIO_DATA_AUG = 2


def compute_class_images_count(base_folder: str, class_name: str):
    return sum((1 for _ in os.listdir(f'{base_folder}/{class_name}')))


def compute_all_classes_images_count(base_folder: str):
    return sum((compute_class_images_count(base_folder, c) for c in destination_classes))


def compute_train_images_count():
    return compute_all_classes_images_count(DATASET_TRAIN_DIRECTORY)


def compute_val_images_count():
    return compute_all_classes_images_count(DATASET_VAL_DIRECTORY)


def compute_total_images_count():
    return compute_val_images_count() + compute_train_images_count()


def compute_class_weight():
    class_weight = {}
    for c in destination_classes:
        class_weight[int(c)] = compute_class_images_count(DATASET_TRAIN_DIRECTORY, c)
        class_weight[int(c)] += compute_class_images_count(DATASET_VAL_DIRECTORY, c)

    # Recuperation de la classe comportortant le moins de data
    key_min = min(class_weight.keys(), key=(lambda k: class_weight[k]))
    to_divide = class_weight[key_min]

    for c in destination_classes:
        class_weight[int(c)] /= to_divide

    return class_weight


## Utils Functions

In [4]:
def save_spectrogramm(d, s, picture_path):
    xx, frequency, bins, im = plt.specgram(d, Fs=s)
    plt.axis('off')
    plt.savefig(picture_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    image = Image.open(picture_path)
    image.convert('RGB').resize((IMAGE_WIDTH, IMAGE_HEIGHT)).save(picture_path)


def save_mel_spectrogramm(d, s, picture_path):
    spec = np.abs(librosa.stft(np.array(d), hop_length=512))
    spec = librosa.amplitude_to_db(spec, ref=np.max)
    librosa.display.specshow(spec, sr=s, cmap='magma')
    plt.axis('off')
    plt.savefig(picture_path, bbox_inches='tight', pad_inches=0)
    image = Image.open(picture_path)
    image.convert('RGB').resize((IMAGE_WIDTH, IMAGE_HEIGHT)).save(picture_path)


def save_random_brig(d, s, picture_patch):
    spec = np.abs(librosa.stft(np.array(d), hop_length=512))
    spec = librosa.amplitude_to_db(spec, ref=np.max)
    librosa.display.specshow(spec, sr=s, cmap='magma')
    plt.axis('off')
    plt.savefig(picture_patch, bbox_inches='tight', pad_inches=0)
    img = Image.open(picture_patch)
    imgArray = np.asarray(img)
    img2 = tf.image.random_brightness(imgArray,0.2)
    finalImag = tf.keras.preprocessing.image.array_to_img(img2)
    finalImag.save(picture_patch)

def load_data(path):
    labels = np.zeros(0, dtype=np.float32)
    data = np.zeros((0, IMAGE_HEIGHT, IMAGE_WIDTH, 4), dtype=np.float32)
    for _, directories, _ in os.walk(path):
        for directory in directories:
            directory_path = os.path.join(path, directory)
            for file in os.listdir(directory_path):
                labels = np.append(labels, int(directory))
                spectro_image = image.imread(os.path.join(directory_path, file))
                spectro_image = np.expand_dims(spectro_image, axis=0)
                data = np.concatenate((data, spectro_image), axis=0)
    return data, labels


def split_array(data_to_split, percent):
    if percent > 1:
        raise Exception("percent parameter need to be between 0 and 1")

    percent_indice = int(len(data_to_split) * percent)
    return np.array([data_to_split[i] for i in range(percent_indice)]), \
           np.array([data_to_split[i] for i in range(percent_indice, len(data_to_split))])


def build_x_y(x, y):
    return x, tf.keras.utils.to_categorical(y, len_classes)


def count_csv_lines(path):
    with open(path, mode='r') as file:
        reader = csv.DictReader(file)
        count = 0
        for _ in reader:
            count += 1
        return count


def plot_all_logs(logs):
    print(logs)
    metrics = ['loss', 'val_loss', 'categorical_accuracy', 'val_categorical_accuracy']
    for metric in metrics:
        for log in logs:
            y_coords = log['value'].history[metric]
            x_coords = list(range(len(y_coords)))
            plt.plot(x_coords, y_coords)
            plt.title(log['title'] + " - " + datetime.now().strftime("%Hh:%Mm:%Ss") + " - " + metric)
            plt.show()


# Dataset Creation

## Train / Validation

In [5]:
augmentations = [
    Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        AddGaussianSNR()
    ]),
    Compose([
        FrequencyMask()
    ])  # ,
    # TODO : AddBackgroundNoise
]

initial_freq = 48000
metadata_inpath = os.path.join(ORIGINAL_DATASET_DIRECTORY, 'train_tp.csv')
audio_inpath = os.path.join(ORIGINAL_DATASET_DIRECTORY, 'train')
number_extract_created = 0


def determine_class_directory(t_min, t_max, current_duration, duration, species_id, is_train):
    class_directory = ""
    dataset_directory = DATASET_TRAIN_DIRECTORY if is_train else DATASET_VAL_DIRECTORY

    if (t_min <= current_duration <= t_max and t_max - current_duration >= MINIMAL_ANIMAL_PRESENCE) or \
            (t_min <= current_duration + duration <= t_max
             and t_max - (current_duration + duration) >= MINIMAL_ANIMAL_PRESENCE) \
            or (current_duration <= t_min and current_duration + duration >= t_max):

        class_directory = os.path.join(dataset_directory, str(species_id))

    elif USE_EMPTY_CLASS and (current_duration + duration <= t_min or current_duration >= t_max):
        class_directory = os.path.join(dataset_directory, str(len_classes - 1))

    return class_directory


def process_data_and_save_spectrogramm(row_data, is_train):
    global number_extract_created
    current_duration = 0
    it = 0
    to_data_aug = 0
    t_min = row_data["t_min"]
    t_max = row_data["t_max"]
    recording_id = row_data["recording_id"]
    row_species = row_data["species_id"]

    data, sample = sf.read(os.path.join(audio_inpath, recording_id + ".flac"))
    end_audio = (len(data) - 1) / sample

    # print(F"processing {recording_id}, species : {row_species} [{t_min},{t_max}] duration({end_audio / initial_freq})")
    while current_duration <= end_audio:

        duration = DURATION_CUT
        create_empty_extract = False

        # if RANDOM_CUT:
        #  duration += random.randint(0, len())

        class_directory = determine_class_directory(t_min, t_max, current_duration, duration, row_species, is_train)
        is_empty_extract = "24" in class_directory

        if is_empty_extract and number_extract_created % RATIO_EMPTY_CLASS == 0:
            create_empty_extract = True

        if class_directory == "":
            current_duration += duration
            continue

        if is_empty_extract and not create_empty_extract:
            current_duration += duration
            number_extract_created += 1
            continue

        # print(F"Current duration : {current_duration} => Class_directory({class_directory}) )")

        extract_path = os.path.join(class_directory, recording_id)
        extract_path_da = os.path.join(class_directory, recording_id)

        max_duration_size = len(data) - 1 if len(data) <= (int((current_duration + duration) * initial_freq)) \
            else (int((current_duration + duration) * initial_freq))

        save_spectrogramm([data[j] for j in range(int(current_duration * initial_freq),
                                                      max_duration_size)],
                              sample,
                              extract_path + "_" + str(it) + ".png")

        if USE_DATA_AUGMENTATION and is_train is False and to_data_aug % RATIO_DATA_AUG == 0:
            new_data = augmentations[to_data_aug % 2](samples=data, sample_rate=sample)
            extract_path += F"_{str(it)}__{to_data_aug}.png"
            extract_path_da += F"_{str(it)}__{to_data_aug}_.png"

            save_spectrogramm([new_data[j] for j in range(int(current_duration * initial_freq),
                                                              max_duration_size)],
                                  sample,
                                  extract_path)
            save_random_brig([new_data[j] for j in range(int(current_duration * initial_freq),
                                                         max_duration_size)],
                             sample,
                             extract_path_da)

            to_data_aug += 1

        current_duration += duration
        it += 1
        number_extract_created += 1

    # print(F"{end_audio - current_duration} >= Minimal duration ?? )")
    if end_audio - current_duration >= MINIMAL_DURATION:
        duration = DURATION_CUT
        row_species = row_data["species_id"]

        class_directory = determine_class_directory(t_min, t_max, current_duration, duration, row_species, is_train)
        if class_directory != "":
            extract_path = os.path.join(class_directory, recording_id)

            max_duration_size = len(data) - 1 if len(data) <= (int(end_audio * initial_freq)) \
                else (int(end_audio * initial_freq))

            save_spectrogramm([data[i] for i in range(int(current_duration * initial_freq)
                                                          , max_duration_size)],
                                  sample,
                                  extract_path + "_r.png")
            number_extract_created += 1


def create_spectro_dataset():
    table_tp = pd.read_csv(metadata_inpath).sort_values("recording_id")

    df_train, df_test, _, _ = train_test_split(table_tp,
                                               table_tp["species_id"],
                                               test_size=validation_split,
                                               random_state=50,
                                               stratify=table_tp["species_id"])
    counter = 0
    for index, row in df_train.iterrows():
        print(F"{counter}/{len(df_train.index)}")
        process_data_and_save_spectrogramm(row, is_train=True)
        counter += 1

    counter = 0
    for index, row in df_test.iterrows():
        print(F"{counter}/{len(df_test.index)}")
        process_data_and_save_spectrogramm(row, is_train=False)
        counter += 1

    print('100%')


## Test

In [6]:
test_path = os.path.join(ORIGINAL_DATASET_DIRECTORY, 'test')
initial_freq = 48000


def create_test_spectro_dataset():
    one_percent = int(sum([len(files) for r, d, files in os.walk(test_path)]) / 100)
    percent = 0
    line_count = 0
    for file in os.listdir(test_path):
        file_path = (os.path.join(test_path, file))
        data, sample = sf.read(file_path)
        end_audio = (len(data) - 1) / sample

        directory_music = os.path.join(DATASET_TEST_DIRECTORY, file.replace(".flac", ""))
        if not os.path.isdir(directory_music):
            os.mkdir(directory_music)
        new_file_path = (os.path.join(directory_music, file.replace(".flac", "")))

        if line_count % one_percent == 0:
            if percent % PERCENT_PRINT == 0:
                print(str(percent) + "%")
            percent += 1

        duration = DURATION_CUT
        # if RANDOM_CUT:
        #  duration += random.randint(0, len())

        current_duration = 0
        it = 0
        while current_duration <= end_audio:
            max_duration_size = len(data) - 1 if len(data) <= (int((current_duration + duration) * initial_freq)) \
                else (int((current_duration + duration) * initial_freq))

            save_spectrogramm([data[j] for j in range(int(current_duration * initial_freq),
                                                      max_duration_size)],
                              sample, new_file_path + "_" + str(it) + ".png")
            current_duration += duration
            it += 1

        if end_audio - current_duration >= MINIMAL_DURATION:
            max_duration_size = len(data) - 1 if len(data) <= (int(end_audio * initial_freq)) \
                else (int(end_audio * initial_freq))

            save_spectrogramm([data[i] for i in range(int(current_duration * initial_freq)
                                                      , max_duration_size)],
                              sample, new_file_path + "_r.png")

        line_count += 1

    print("100%")

## Cleaning and creation of dataset

In [7]:
dataset_type = [
    "val_train",
    "test"
]

def clean_dataset(dataset):
    if dataset == dataset_type[0]:
        for c in destination_classes:
            folder = f'{DATASET_TRAIN_DIRECTORY}/{c}'
            if os.path.exists(folder):
                for filename in os.listdir(folder):
                    os.remove(f'{folder}/{filename}')

            folder = f'{DATASET_VAL_DIRECTORY}/{c}'
            if os.path.exists(folder):
                for filename in os.listdir(folder):
                    os.remove(f'{folder}/{filename}')
    else:
        for folder in os.listdir(DATASET_TEST_DIRECTORY):
            shutil.rmtree(os.path.join(DATASET_TEST_DIRECTORY, folder), ignore_errors=True)


def clean_or_create_empty_folder():
    empty_paths = [os.path.join(DATASET_VAL_DIRECTORY, "24"), os.path.join(DATASET_TRAIN_DIRECTORY, "24")]

    if USE_EMPTY_CLASS:
        for path in empty_paths:
            if not os.path.exists(path):
                os.mkdir(path)
    else:
        for path in empty_paths:
            if os.path.exists(path):
                os.rmdir(path)


def clean_and_create_dataset(dataset):
    if dataset == dataset_type[0]:
        if not os.path.isdir(DATASET_DIRECTORY):
            os.mkdir(DATASET_DIRECTORY)
        if not os.path.isdir(DATASET_TRAIN_DIRECTORY):
            os.mkdir(DATASET_TRAIN_DIRECTORY)
        if not os.path.isdir(DATASET_VAL_DIRECTORY):
            os.mkdir(DATASET_VAL_DIRECTORY)
    else:
        if not os.path.isdir(DATASET_TEST_DIRECTORY):
            os.mkdir(DATASET_TEST_DIRECTORY)

    print(F"Cleaning {dataset} dataset...")
    clean_dataset(dataset)

    clean_or_create_empty_folder()

    print(F"Creating {dataset} dataset...")

    if dataset == dataset_type[0]:
        create_spectro_dataset()
    else:
        create_test_spectro_dataset()



# Change dataset_type pour generer test_val ou test
# 0 == train & val
# 1 == test
clean_and_create_dataset(dataset_type[0])

Cleaning val_train dataset...
Creating val_train dataset...
0/851
1/851
2/851
3/851
4/851
5/851
6/851
7/851
8/851
9/851
10/851
11/851
12/851
13/851
14/851
15/851
16/851
17/851
18/851
19/851
20/851
21/851
22/851
23/851
24/851
25/851
26/851
27/851
28/851
29/851
30/851
31/851
32/851
33/851
34/851
35/851
36/851
37/851
38/851
39/851
40/851
41/851
42/851
43/851
44/851
45/851
46/851
47/851
48/851
49/851
50/851
51/851
52/851
53/851
54/851
55/851
56/851
57/851
58/851
59/851
60/851
61/851
62/851
63/851
64/851
65/851
66/851
67/851
68/851
69/851
70/851
71/851
72/851
73/851
74/851
75/851
76/851
77/851
78/851
79/851
80/851
81/851
82/851
83/851
84/851
85/851
86/851
87/851
88/851
89/851
90/851
91/851
92/851
93/851
94/851
95/851
96/851
97/851
98/851
99/851
100/851
101/851
102/851
103/851
104/851
105/851
106/851
107/851
108/851
109/851
110/851
111/851
112/851
113/851
114/851
115/851
116/851
117/851
118/851
119/851
120/851
121/851
122/851
123/851
124/851
125/851
126/851
127/851
128/851
129/851
130/851
13

194/365
195/365
196/365
197/365
198/365
199/365
200/365
201/365
202/365
203/365
204/365
205/365
206/365
207/365
208/365
209/365
210/365
211/365
212/365
213/365
214/365
215/365
216/365
217/365
218/365
219/365
220/365
221/365
222/365
223/365
224/365
225/365
226/365
227/365
228/365
229/365
230/365
231/365
232/365
233/365
234/365
235/365
236/365
237/365
238/365
239/365
240/365
241/365
242/365
243/365
244/365
245/365
246/365
247/365
248/365
249/365
250/365
251/365
252/365
253/365
254/365
255/365
256/365
257/365
258/365
259/365
260/365
261/365
262/365
263/365
264/365
265/365
266/365
267/365
268/365
269/365
270/365
271/365
272/365
273/365
274/365
275/365
276/365
277/365
278/365
279/365
280/365
281/365
282/365
283/365
284/365
285/365
286/365
287/365
288/365
289/365
290/365
291/365
292/365
293/365
294/365
295/365
296/365
297/365
298/365
299/365
300/365
301/365
302/365
303/365
304/365
305/365
306/365
307/365
308/365
309/365
310/365
311/365
312/365
313/365
314/365
315/365
316/365
317/365
318/365


# Submission functions

In [8]:
def average(predicts, *args):
    sum_classes = np.zeros(length_classes, dtype=np.float32)
    for predict in predicts:
        print(predict)
        for i in range(length_classes):
            sum_classes[i] += predict[i]
    return sum_classes / len(predicts)

In [9]:
def random(predicts, *args):
    return predicts[random.randint(0, len(predicts) - 1)]

In [10]:
def higher_than(predicts, limit):
    limit = limit[0]
    higher_prediction = np.zeros(length_classes, dtype=np.float32)
    for predict in predicts:
        for i in range(length_classes):
            if predict[i] >= limit:
                higher_prediction[i] += 1
    total = sum(higher_prediction)
    if total == 0:
        return higher_than(predicts, (limit - limit / 3,))
    return higher_prediction / total

In [11]:
def highest_value(predicts, *args):
    for predict in predicts:
        for i in range(length_classes):
            if predict[i] >= 0.6:
                return

In [12]:
def distribute_empty(pred):
    return_array = np.zeros(len(pred))
    to_distribute = pred[len(pred) - 1]
    for p in range(length_classes):
        return_array[p] = pred[p] + (to_distribute / length_classes)
    return return_array

In [13]:
def predict_and_save_in_submission(model: Model, func, *args):
    with open(os.path.join(DATASET_DIRECTORY, "submission.csv"), mode='w', newline='') as output_csv_file:
        writer = csv.writer(output_csv_file)
        writer.writerow(["recording_id", "s0", "s1", "s2", "s3", "s4", "s5",
                         "s6", "s7", "s8", "s9", "s10", "s11", "s12", "s13", "s14",
                         "s15", "s16", "s17", "s18", "s19", "s20", "s21", "s22", "s23"])
        one_percent = int(sum([len(files) for r, d, files in os.walk(DATASET_TEST_DIRECTORY)]) / 100)
        percent = 0
        line_count = 0
        for _, directories, _ in os.walk(DATASET_TEST_DIRECTORY):
            for directory in directories:
                directory_path = os.path.join(DATASET_TEST_DIRECTORY, directory)
                predictions = np.zeros((0, len_classes), dtype=np.float32)

                for file in os.listdir(directory_path):
                    file_path = os.path.join(directory_path, file)
                    spectro_image = image.imread(file_path)
                    spectro_image = np.expand_dims(spectro_image, axis=0)

                    model_prediction = model.predict(spectro_image)

                    # if USE_EMPTY_CLASS and model_prediction[0][len_classes - 1] >= PRED_EMPTY_IGNORE_EXTRACT:
                    #    model_prediction[0] = np.zeros(len_classes, dtype=np.float32)
                    if USE_EMPTY_CLASS:
                        model_prediction[0] = distribute_empty(model_prediction[0])

                    predictions = np.concatenate(
                        (predictions,
                         model_prediction),
                        axis=0)

                    if line_count % one_percent == 0:
                        if percent % PERCENT_PRINT == 0:
                            print(str(percent) + "%")
                        percent += 1
                    line_count += 1

                if func is None:
                    writer.writerow(np.insert(predictions[0].astype(np.str), 0, directory, axis=0))
                else:
                    writer.writerow(np.insert(func(predictions, args).astype(np.str), 0, directory, axis=0))

# Mixup DataGenerator

In [14]:
class MixupImageDataGenerator():
    def __init__(self, generator, directory, batch_size, img_height, img_width, alpha=0.2, subset=None):
        """Constructor for mixup image data generator.

        Arguments:
            generator {object} -- An instance of Keras ImageDataGenerator.
            directory {str} -- Image directory.
            batch_size {int} -- Batch size.
            img_height {int} -- Image height in pixels.
            img_width {int} -- Image width in pixels.

        Keyword Arguments:
            alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
            subset {str} -- 'training' or 'validation' if validation_split is specified in
            `generator` (ImageDataGenerator).(default: {None})
        """

        self.batch_index = 0
        self.batch_size = batch_size
        self.alpha = alpha

        # First iterator yielding tuples of (x, y)
        self.generator1 = generator.flow_from_directory(directory,
                                                        target_size=(
                                                            img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        subset=subset)

        # Second iterator yielding tuples of (x, y)
        self.generator2 = generator.flow_from_directory(directory,
                                                        target_size=(
                                                            img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        subset=subset)

        # Number of images across all classes in image directory.
        self.n = self.generator1.samples

    def reset_index(self):
        """Reset the generator indexes array.
        """

        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        # round up
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        """Get number of steps per epoch based on batch size and
        number of images.

        Returns:
            int -- steps per epoch.
        """

        return self.n // self.batch_size

    def __next__(self):
        """Get next batch input/output pair.

        Returns:
            tuple -- batch of input/output pair, (inputs, outputs).
        """

        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0

        # random sample the lambda value from beta distribution.
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)

        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        # Get a pair of inputs and outputs from two iterators.
        X1, y1 = self.generator1.next()
        X2, y2 = self.generator2.next()

        # Perform the mixup.
        X = X1 * X_l + X2 * (1 - X_l)
        y = y1 * y_l + y2 * (1 - y_l)
        return X, y

    def __iter__(self):
        while True:
            yield next(self)

# Starting

In [25]:
train_size = compute_train_images_count()
val_size = compute_val_images_count()
class_w = compute_class_weight()

2270
990


In [16]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

# Base model creation

In [17]:
def create_base_model(add_custom_layers_func) -> Model:
    m = Sequential()
    add_custom_layers_func(m)

    m.add(Flatten())
    m.add(tf.keras.layers.Dense(len_classes, tf.keras.activations.softmax))

    m.compile(optimizer=tf.keras.optimizers.SGD(lr=ref_lr / ref_batch_size * batch_size),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=["categorical_accuracy"])

    return m

# MLP

In [18]:
def add_mlp_layers(model):
    model.add(tf.keras.layers.Flatten())
    for _ in range(5):
        model.add(tf.keras.layers.Dense(2048, activation=tf.keras.activations.linear))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation(activation=tf.keras.activations.tanh))

# ConvNet

In [31]:
def add_convnet(model):
    model.add(tf.keras.layers.Reshape((IMAGE_WIDTH, IMAGE_HEIGHT, 4)))

    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.keras.activations.tanh,
                                     kernel_regularizer=tf.keras.regularizers.l2(KERNEL_REGULARIZERS)))
    model.add(tf.keras.layers.MaxPool2D())

    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.keras.activations.tanh,
                                     kernel_regularizer=tf.keras.regularizers.l2(KERNEL_REGULARIZERS)))
    model.add(tf.keras.layers.MaxPool2D())

    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.keras.activations.tanh,
                                     kernel_regularizer=tf.keras.regularizers.l2(KERNEL_REGULARIZERS)))
    model.add(tf.keras.layers.MaxPool2D())

# Model Train

In [38]:
def train_model(m: Model, x_iterator, y_iterator):
    log = m.fit(
        x_iterator,
        validation_data=y_iterator,
        steps_per_epoch=train_size // batch_size,
        validation_steps=val_size // batch_size,
        epochs=epch,
        class_weight=class_w
    )
    return log

# Dataset Iterator

In [39]:
def create_dataset_iterator(base_folder: str, size: int):
    def inner_func():
        return tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(base_folder,
                                                                                                      target_size=(
                                                                                                          IMAGE_WIDTH,
                                                                                                          IMAGE_HEIGHT),
                                                                                                      color_mode='rgba',
                                                                                                      batch_size=1)

    return (tf.data.Dataset.from_generator(inner_func,
                                           output_types=(tf.float32, tf.float32),
                                           output_shapes=(
                                               (1, *(IMAGE_WIDTH, IMAGE_HEIGHT), 4),
                                               (1, len_classes)
                                           )
                                           )
            .take(size)
            .unbatch()
            .batch(batch_size)
            .cache(f'{base_folder}/cache')
            .repeat()
            .as_numpy_iterator()
            )

# ConvNet execution

In [40]:
model = create_base_model(add_convnet)

### Logs

In [42]:
all_logs = [
    {"value": train_model(model,
        create_dataset_iterator(DATASET_TRAIN_DIRECTORY, train_size),
        create_dataset_iterator(DATASET_VAL_DIRECTORY, val_size)),
    "title": "add_convnet"}
]

plot_all_logs(all_logs)

Found 2270 images belonging to 25 classes.
Epoch 1/100
1135/1135 [==============================] - ETA: 0s - loss: 3426.1724 - categorical_accuracy: 0.1137Found 990 images belonging to 25 classes.
Found 2270 images belonging to 25 classes.
1135/1135 [==============================] - 101s 89ms/step - loss: 3426.1724 - categorical_accuracy: 0.1137 - val_loss: 974.7761 - val_categorical_accuracy: 0.0303
Epoch 2/100
1135/1135 [==============================] - 87s 77ms/step - loss: 3450.8516 - categorical_accuracy: 0.1031 - val_loss: 2675.6775 - val_categorical_accuracy: 0.2586
Epoch 3/100
1135/1135 [==============================] - 51s 45ms/step - loss: 3379.4539 - categorical_accuracy: 0.1053 - val_loss: 2491.5337 - val_categorical_accuracy: 0.2586
Epoch 4/100
1135/1135 [==============================] - 51s 45ms/step - loss: 3474.8933 - categorical_accuracy: 0.1097 - val_loss: 2426.6074 - val_categorical_accuracy: 0.2586
Epoch 5/100
1135/1135 [==============================] - 53s 47

KeyboardInterrupt: 

### Train

In [ ]:
print("[Train] => ")
model.evaluate(create_dataset_iterator(DATASET_TRAIN_DIRECTORY, train_size),
                steps=train_size // batch_size)

### Validation

In [ ]:
print("[Validation] => ")
model.evaluate(create_dataset_iterator(DATASET_VAL_DIRECTORY, val_size),
                steps=val_size // batch_size)

### Save prediction on test dataset

In [ ]:
print("Sauvegarde des prédictions sur le jeu de test : ")
predict_and_save_in_submission(model, higher_than, 0.4)

# Efficient Net

### Callbacks

In [ ]:
def get_callbacks():
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                                  verbose=1, mode='auto', min_delta=0.0001,
                                  cooldown=0, min_lr=0)

    early_stopping = EarlyStopping(monitor="val_loss", patience=7, verbose=1, restore_best_weights=True)

    model_cp = ModelCheckpoint(WEIGHT_FILE_NAME,
                               save_best_only=True,
                               save_weights_only=True,
                               monitor='val_loss',
                               mode='min', verbose=1)

    return [early_stopping, model_cp, reduce_lr]

### Create model

In [ ]:
def create_efficient_net_models():
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))

    m = Sequential([
        EfficientNet(include_top=False, weights='imagenet', input_tensor=inputs),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.GlobalAveragePooling2D(name="avg_pool"),
        layers.BatchNormalization(),
        layers.Dropout(dropout, name="top_dropout"),
        layers.Dense(len_classes, activation="tanh", name="pred")
    ])
    m.compile(loss=losses.CategoricalCrossentropy(),
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])
    return m

### Train model

In [ ]:
def train_model(m, x_iterator, y_iterator):
    log = m.fit(x_iterator,
                validation_data=y_iterator,
                steps_per_epoch=x_iterator.get_steps_per_epoch(),
                validation_steps=y_iterator.samples // batch_size,
                epochs=epch,
                callbacks=get_callbacks())
    return log

# Execution de l'Efficient Net

### Model creation

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from efficientnet.tfkeras import EfficientNetB1 as EfficientNet

In [ ]:
print(F"Creating model...")
model = create_efficient_net_models()

In [ ]:
input_imgen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_generator = MixupImageDataGenerator(generator=input_imgen,
                                              directory=DATASET_TRAIN_DIRECTORY,
                                              batch_size=batch_size,  # To verify maybe error
                                              img_height=IMAGE_HEIGHT,
                                              img_width=IMAGE_WIDTH)

In [ ]:
 validation_generator = input_imgen.flow_from_directory(directory=DATASET_VAL_DIRECTORY,
                                                           target_size=(
                                                               IMAGE_WIDTH,
                                                               IMAGE_HEIGHT),
                                                           batch_size=batch_size,  # To verify maybe error
                                                           class_mode="categorical",
                                                           shuffle=True)

In [ ]:
print('training steps: ', train_generator.get_steps_per_epoch())
print('validation steps: ', validation_generator.samples // batch_size)

### Train

In [ ]:
print(F"Training model...")
all_logs = [
    {"value": train_model(model,
                            train_generator,
                            validation_generator),
     "title": F"efficient_net"}
]
plot_all_logs(all_logs)

### Evaluate

In [ ]:
print(F"Evaluation du model...")
validation_generator._set_index_array()
model.evaluate(validation_generator,
                steps=validation_generator.samples // batch_size)
predict_and_save_in_submission(model, higher_than, 0.35)